In [50]:
%load_ext autoreload
%autoreload 2

# Analyse des accidents de la route en France  
## Introduction
Les accidents de la route constituent un enjeu majeur de santé publique en France, avec plusieurs milliers de morts et de blessés graves chaque année. Leur compréhension repose sur l’analyse conjointe de facteurs temporels, environnementaux, infrastructurels et humains.

Ce projet s’appuie sur les données officielles des accidents corporels de la circulation en France, mises à disposition par les autorités publiques. Ces données décrivent, pour chaque accident, ses caractéristiques temporelles, géographiques, environnementales ainsi que certaines informations sur les usagers et les véhicules impliqués.

L’objectif de ce travail est de proposer une analyse statistique exploratoire des accidents de la route, en mettant l’accent sur les facteurs associés à la **gravité des accidents**, tout en tenant compte des limites inhérentes aux données disponibles.


In [1]:
!pip install -r requirements.txt

# modules 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import re


# Fonctions
from urllib import request
from modules import module_BAAC
from modules import graph_baac


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: C:\Users\gmass\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
     ----------------------- -------------- 153.6/250.9 kB 4.6 MB/s eta 0:00:01
     -------------------------------------- 250.9/250.9 kB 3.9 MB/s eta 0:00:00


### Présentation des données

L’analyse repose principalement sur les jeux de données des accidents de la route survenus en France de 2005 à 2024, ces données sont issues d'API publiques (data.gouv.fr, INSEE et transport.data.gouv.fr). Chaque ligne correspond à un accident et décrit notamment :

- la date et l’heure de l’accident, 
- la localisation géographique (coordonnées, département, commune), 
- les conditions environnementales (luminosité, conditions atmosphériques, état de la chaussée),
- le sexe du conducteur, 
- le nombre de véhicules et d’usagers impliqués, 
- la gravité de l’accident

Le jeux de données antérieurs à 2024 seront mobilisés ponctuellement pour étudier les évolutions temporelles à long terme. Chaque jeu de donnée est intitulé 'BAAC_table_année' et le jeu de donnée regroupant tous les accidents depuis 2005 est intitulé 'BAAC_table'. 


In [2]:
# API Bases de données annuelles des accidents corporels de la circulation routière – 2005 à 2024
BAAC_url = "https://www.data.gouv.fr/api/1/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/"

# API Bases de données sur le parc automobile français
parc_auto_url = "https://www.data.gouv.fr/api/1/datasets/parc-automobile/"

# API INSEE (statistiques socio-économiques)
insee_url = "https://api.insee.fr/series/BDM/V1/data"

# API transport.data.gouv.fr (mobilité et trafic)
transport_url = "https://transport.data.gouv.fr/api/datasets"

In [3]:
BAAC_resources = module_BAAC.get_resources(BAAC_url)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = 300
BAAC_resources[["description", "url"]].head()

,description,url
0,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2024.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100210/2024.csv
1,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2023.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-142813/2023.csv
2,Table des caractéristiques des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115900/caract-2024.csv
3,Table des lieux des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115812/lieux-2024.csv
4,Table des véhicules impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100240/vehicules-2024.csv
5,Table des usagers impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115506/usagers-2024.csv
6,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2022.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-142722/2022.csv
7,Table des usagers impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153328/usagers-2023.csv
8,Table des véhicules impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153253/vehicules-2023.csv
9,Table des lieux des accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153219/lieux-2023.csv


In [4]:
selected_BAAC_table = module_BAAC.select_baac_tables(BAAC_resources)
selected_BAAC_table.head(2)

,description,url,table,year
0,Table des caractéristiques des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115900/caract-2024.csv,caracteristiques,2024
1,Table des lieux des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115812/lieux-2024.csv,lieux,2024
2,Table des véhicules impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100240/vehicules-2024.csv,vehicules,2024
3,Table des usagers impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115506/usagers-2024.csv,usagers,2024
4,Table des usagers impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153328/usagers-2023.csv,usagers,2023


In [5]:
BAAC_table = module_BAAC.build_baac_dataframe(selected_BAAC_table)
BAAC_table.head()

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,...,dep,year,id_vehicule_x,secu1,secu2,secu3,id_vehicule_y,motor,vma,id_usager
0,200500000001,1,1,4,1,1,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200500000001,1,1,4,1,1,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200500000001,1,1,3,2,3,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200500000001,1,1,3,2,3,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200500000001,2,2,1,1,0,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Choix des variables conservées

La table principale BAAC_table comporte 61 variables, qui ne sont pas toutes pertinentes dans le cadre de cette étude. Nous avons décidé de conserver les variables suivantes : 'grav' (gravité de l'accident), 'year', 'mois', 'jour', 'hrmn' (→ heure), 'lat', 'long', 'dep', 'agg' (agglomération ou non), 'vma' (vitesse limite), 'nbv', 'catr', 'circ', 'lum', 'atm', 'surf' (paramètres physiques), 'sexe', 'an_nais' (année de naissance). Sur certaines tables, 'hrmn' est un string de la forme 'HH:MM' (avant 2019), sur d'autres 'hrmn' est de la fome HHMM, nous avons donc remplacé 'hmrn' par une variable 'hour' en conservant seulement l'heure de l'accident HH. 

In [55]:
BAAC_table['hour'] = BAAC_table['hrmn'].apply(graph_baac.hrmn_to_hour)

variables_conservees = ["grav", "year", "mois", "jour", "hour", "lat", "long", "dep", "agg",
 "vma", "nbv", "catr", "circ", "lum", "atm", "surf", "sexe", "an_nais"]

BAAC_table = BAAC_table[variables_conservees]


KeyError: 'hrmn'

On crée ensuite les tables associées à chaque année à partir de la table principale : 

In [53]:
for i in range(2005, 2025):
    globals()[f'BAAC_table_{i}'] = BAAC_table.loc[BAAC_table['year'] == i].reset_index(drop=True)


Index(['grav', 'year', 'mois', 'jour', 'hour', 'lat', 'long', 'dep', 'agg',
       'vma', 'nbv', 'catr', 'circ', 'lum', 'atm', 'surf', 'sexe', 'an_nais'],
      dtype='object')

### Valeurs manquantes
 
La table principale étant issue de la concaténation de vingt sous-tables, il est nécessaire de vérifier la proportion de mesures disponibles pour chaque variable et chaque année, afin d’éviter d’effectuer des analyses sur des valeurs manquantes (NaN).

Un tableau a été élaboré pour représenter, année par année, la proportion de valeurs effectivement renseignées pour chaque variable. Il permet par exemple de constater que que la variable 'vma' n’a été introduite qu’à partir de 2019. 

In [54]:
l = ["grav", "year", "mois", "jour", "hour", "lat", "long", "dep", "agg",
 "vma", "nbv", "catr", "circ", "lum", "atm", "surf", "sexe", "an_nais"]     #variables conservées

years = range(2005, 2025)
df_proportion = pd.DataFrame(index=years, columns=l)

for i in years:
    table = globals()[f'BAAC_table_{i}']
    for var in l:
        df_proportion.loc[i, var] = 1 - table[var].isna().sum() / len(table)

df_proportion = df_proportion.astype(float)

df_proportion = df_proportion.loc[:, (df_proportion != 1).any(axis=0)]

df_proportion

,mois,jour,hour,lat,long,dep,agg,vma,nbv,catr,circ,lum,atm,surf,an_nais
2005,1.0,1.0,1.0,0.323421,0.323421,1.0,1.0,0.0,1.000000,0.999995,1.000000,1.0,1.000000,1.000000,0.996738
2006,1.0,1.0,1.0,0.328526,0.328526,1.0,1.0,0.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,0.995026
2007,1.0,1.0,1.0,0.294926,0.294926,1.0,1.0,0.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,0.998316
2008,1.0,1.0,1.0,0.298939,0.298939,1.0,1.0,0.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,0.999736
2009,1.0,1.0,1.0,0.466751,0.466751,1.0,1.0,0.0,0.990449,1.000000,0.999217,1.0,0.999926,0.999095,0.999920
2010,1.0,1.0,1.0,0.466228,0.466228,1.0,1.0,0.0,0.998108,1.000000,0.999136,1.0,0.999986,0.999125,0.999632
2011,1.0,1.0,1.0,0.467052,0.467052,1.0,1.0,0.0,0.997440,1.000000,0.998481,1.0,0.999808,0.998268,0.999528
2012,1.0,1.0,1.0,0.464061,0.463985,1.0,1.0,0.0,0.996075,1.000000,0.996846,1.0,0.999954,0.995669,0.999913
2013,1.0,1.0,1.0,0.496620,0.496620,1.0,1.0,0.0,0.998394,1.000000,0.998992,1.0,0.999769,0.998695,0.999645
2014,1.0,1.0,1.0,0.519430,0.519430,1.0,1.0,0.0,0.997543,1.000000,0.998568,1.0,0.999763,0.997635,0.999863


## Problématique

Si la fréquence et la localisation des accidents de la route sont relativement bien documentées, la compréhension des **facteurs associés à la gravité des accidents** reste un enjeu central pour l’analyse et la prévention du risque routier.

La problématique principale de ce projet est donc la suivante :

> *Dans quelles conditions temporelles, environnementales et infrastructurelles les accidents de la route sont-ils les plus graves ?*

Cette question sera abordée à travers une analyse descriptive, exploratoire et multivariée des données, sans prétendre établir des relations causales strictes.


## Enjeux et choix méthodologiques

Plusieurs enjeux méthodologiques structurent ce travail :

Premièrement, les données sont observationnelles et ne permettent pas d’identifier des relations causales directes. De plus les variables territoriales agrégées (régions, départements) sont peu nombreuses au regard du nombre d’accidents observés, ce qui limite leur exploitation dans des modèles explicatifs.

En conséquence, l’analyse privilégie une approche descriptive pour dresser un état des lieux des accidents, une analyse exploratoire multivariée (ACP) pour comprendre la structure des relations entre variables ainsi que des modèles de régression visant à identifier les variables **associées** à la gravité des accidents.


## Organisation du notebook

Le notebook est structuré comme suit :

I. Analyse descriptive des accidents de la route en 2024  
II. Analyse des dynamiques temporelles à long terme (2005–2024)  
III. Analyse exploratoire multivariée (ACP)  
IV. Analyse des facteurs associés à la gravité des accidents  
V. Discussion des résultats et limites  
VI. Conclusion

Chaque partie combine des visualisations graphiques et des analyses statistiques adaptées aux objectifs poursuivis.


## I. Analyse descriptive des accidents de la route

In [77]:
graph_baac.plot_crashes_heatmap(BAAC_table_2024)

AttributeError: Can only use .str accessor with string values!

In [75]:
[(globals()[f'BAAC_table_{i}'][['lat', 'long']].head(1)) for i in range(2005, 2025)]

[       lat     long
 0  5051500  0294400,
    lat long
 0  NaN  NaN,
        lat     long
 0  5051700  0291000,
        lat     long
 0  4515400  -082600,
   lat long
 0   0    0,
        lat     long
 0  0000000  0000000,
          lat      long
 0  5053589.0  295262.0,
        lat     long
 0  5052928  0293643,
        lat     long
 0  5051199  0290046,
        lat     long
 0  0000000  0000000,
    lat  long
 0  0.0   0.0,
    lat  long
 0  0.0   0.0,
        lat     long
 0  5051326  0292191,
        lat     long
 0  5055737  0294992,
         lat     long
 0  48.89621  2.47012,
         lat     long
 0  48.70535  2.43841,
          lat      long
 0  44.038958  4.348022,
    lat  long
 0  NaN   NaN,
          lat      long
 0  48.866386  2.323471,
         lat     long
 0  47.56277  6.75832]